Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [12]:
#Deep Neural Network Model
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relu = tf.nn.relu(logits1)
    
  weights2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits2 = tf.matmul(relu, weights2) + biases2
  loss1 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))
  l2Loss = loss1 + (tf.nn.l2_loss(weights1)/(2*batch_size)) + (tf.nn.l2_loss(weights2)/(2*batch_size))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(l2Loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, l2Loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 1544.409180
Minibatch accuracy: 10.2%
Validation accuracy: 29.6%
Minibatch loss at step 500: 172.746567
Minibatch accuracy: 83.6%
Validation accuracy: 81.7%
Minibatch loss at step 1000: 24.707052
Minibatch accuracy: 83.6%
Validation accuracy: 85.0%
Minibatch loss at step 1500: 3.899754
Minibatch accuracy: 90.6%
Validation accuracy: 85.4%
Minibatch loss at step 2000: 1.012410
Minibatch accuracy: 91.4%
Validation accuracy: 85.3%
Minibatch loss at step 2500: 0.659064
Minibatch accuracy: 87.5%
Validation accuracy: 85.3%
Minibatch loss at step 3000: 0.650018
Minibatch accuracy: 85.9%
Validation accuracy: 85.8%
Test accuracy: 92.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [8]:
#Deep Neural Network Model
batch_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relu = tf.nn.relu(logits1)
    
  weights2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits2 = tf.matmul(relu, weights2) + biases2
  loss1 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))
  l2Loss = loss1 + (tf.nn.l2_loss(weights1)/(2*batch_size)) + (tf.nn.l2_loss(weights2)/(2*batch_size))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(l2Loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, l2Loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 466.778046
Minibatch accuracy: 12.7%
Validation accuracy: 28.6%
Minibatch loss at step 500: 122.600449
Minibatch accuracy: 82.7%
Validation accuracy: 83.5%
Minibatch loss at step 1000: 93.157631
Minibatch accuracy: 87.1%
Validation accuracy: 82.9%
Minibatch loss at step 1500: 72.777695
Minibatch accuracy: 85.4%
Validation accuracy: 83.7%
Minibatch loss at step 2000: 56.710712
Minibatch accuracy: 85.8%
Validation accuracy: 84.9%
Minibatch loss at step 2500: 44.074535
Minibatch accuracy: 89.1%
Validation accuracy: 85.1%
Minibatch loss at step 3000: 34.583302
Minibatch accuracy: 89.5%
Validation accuracy: 85.9%
Test accuracy: 92.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [9]:
#Deep Neural Network Model
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relu = tf.nn.relu(logits1)
  drop1 = tf.nn.dropout(relu,0.5)
  weights2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits2 = tf.matmul(drop1, weights2) + biases2
  loss1 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))
  l2Loss = loss1 + (tf.nn.l2_loss(weights1)/(2*batch_size)) + (tf.nn.l2_loss(weights2)/(2*batch_size))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(l2Loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), weights2) + biases2)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, l2Loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 1749.012695
Minibatch accuracy: 14.1%
Validation accuracy: 30.1%
Minibatch loss at step 500: 176.356186
Minibatch accuracy: 82.8%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 24.724424
Minibatch accuracy: 83.6%
Validation accuracy: 84.4%
Minibatch loss at step 1500: 3.997409
Minibatch accuracy: 89.1%
Validation accuracy: 84.4%
Minibatch loss at step 2000: 1.076956
Minibatch accuracy: 91.4%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 0.750437
Minibatch accuracy: 85.9%
Validation accuracy: 84.4%
Minibatch loss at step 3000: 0.760689
Minibatch accuracy: 84.4%
Validation accuracy: 84.7%
Test accuracy: 91.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [72]:
#Deep Neural Network Model
batch_size = 200
layer1_size = 28*28*2
layer2_size = 28*28
layer3_size = 28*28*0.25
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.get_variable('W1', shape=[image_size * image_size, layer1_size], initializer=tf.contrib.layers.xavier_initializer()) 
#tf.Variable(tf.truncated_normal([image_size * image_size, layer1_size]))
  biases1 = tf.Variable(tf.zeros([layer1_size]))
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relu1 = tf.nn.relu6(logits1)
    
  weights2 = tf.get_variable('W2', shape=[layer1_size, layer2_size], initializer=tf.contrib.layers.xavier_initializer()) 
  biases2 = tf.Variable(tf.zeros([layer2_size]))
  logits2 = tf.matmul(relu1, weights2) + biases2
  relu2 =tf.nn.relu6(logits2)
    
  weights3 = tf.get_variable('W3', shape=[layer2_size, layer3_size], initializer=tf.contrib.layers.xavier_initializer()) 
  biases3 = tf.Variable(tf.zeros([layer3_size]))
  logits3 = tf.matmul(relu2, weights3) + biases3

  relu3 =tf.nn.relu6(logits3)
    
  weights4 = tf.get_variable('W4', shape=[layer3_size, num_labels], initializer=tf.contrib.layers.xavier_initializer()) 
  biases4 = tf.Variable(tf.zeros([num_labels]))
  logits4 = tf.matmul(relu3, weights4) + biases4

  # Training computation.
  loss1 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits4, tf_train_labels))
  l2Loss = loss1 + ((tf.nn.l2_loss(weights1)+ tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights4))/(2*batch_size))
    
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(l2Loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits4)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3),weights4) + biases4)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3),weights4) + biases4)

num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, l2Loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.311243
Minibatch accuracy: 17.5%
Validation accuracy: 15.6%
Minibatch loss at step 500: 3.670395
Minibatch accuracy: 81.0%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 3.379979
Minibatch accuracy: 88.0%
Validation accuracy: 82.8%
Minibatch loss at step 1500: 3.252129
Minibatch accuracy: 89.0%
Validation accuracy: 83.7%
Minibatch loss at step 2000: 3.314777
Minibatch accuracy: 84.5%
Validation accuracy: 84.2%
Minibatch loss at step 2500: 3.181674
Minibatch accuracy: 85.5%
Validation accuracy: 84.6%
Minibatch loss at step 3000: 3.158865
Minibatch accuracy: 85.5%
Validation accuracy: 84.8%
Minibatch loss at step 3500: 2.970549
Minibatch accuracy: 88.5%
Validation accuracy: 85.2%
Minibatch loss at step 4000: 3.098881
Minibatch accuracy: 85.5%
Validation accuracy: 85.4%
Minibatch loss at step 4500: 2.924874
Minibatch accuracy: 85.5%
Validation accuracy: 85.6%
Minibatch loss at step 5000: 2.795087
Minibatch accuracy: 89.0%
Validation accurac